# INDEX
1. Introduction
2. Data
3. Preprocessing
4. Training & Result
6. Discuss



# 1. INTRODUCTION

We executed classification by improving [Simple Keras FastText](https://www.kaggle.com/nzw0301/simple-keras-fasttext-val-loss-0-31) model.
we've got significantly high accuracy by improving baseline model by additional feature engineering method(multi n-gram. text mining methods...) & collecting more data.

During these process, we had ambitions to utilizing and analyzing the concept of author identification with large corpus of data. 

+ Luckily, we obtained  [Toronto Book Corpus](http://yknzhu.wixsite.com/mbweb) (containing lot of books with several authors), so we could check our hypothesis: ’Can we recommend similar author based on one’s sentence or paragraph?’

According to our competition and great kernels, many contributors proved that a sentence can be classified to unique author. 
We interpreted the result of classification as ‘unique writing style of authors’. 

Let’s check out if there exists some insights base on the Author and Theme classification with Toronto Book Corpus  dataset.



# 2. Additional Data

Our data is based on  [Toronto Book Corpus](http://yknzhu.wixsite.com/mbweb).

We extracted 11,038 books & 16 different genres from corpus. 

We decided to consider only 10 out of 16 genres: We excluded (Themes, Young_Adult,New_Adult, Historical , Other,Vampires) from 16 genres (Adventure, Horror, Mystery, Romance, Themes, Young_Adult,Fantasy ,Humor,New_Adult,Science_fiction,Thriller,Historical , Literature, Other, Teen ,Vampires) because the data of those topic were less then other genre.



# 3. Preprocessing Data

We found that there might be no clue for author in some of the books, so we checked if we can access book author or not. We extracted the books author by using Regular Expression.

We covered two simple rule covers by regular expression :
    1. There is a sentence starting with 'by' or 'By' in the book
    2. There is a sentence starting with 'Copyright' in the book
In this way, we obtained 2,711 books out of total 11,038 books and finally 1788 authors.


## 3-a. How many books are exist in each genre
 



In [ ]:
import pickle
import matplotlib.pyplot as plt
import sys
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

D = { 'Adventure': 284, 'Fantasy': 312, 'Horror': 209, 'Humor': 202, 'Literature': 234,
    'Mystery': 242, 'Romance': 304, 'SF': 350, 'Teen': 286, 'Thriller': 288 }

plt.bar(range(len(D)), D.values(), align='center')
plt.xticks(range(len(D)), D.keys())
plt.title("Books by Topic")
plt.show()

total = 0

for key in D:
    total += D[key]

print( "Total: %d"%(total) )

## 3-b. How many books an Author wrote

In [ ]:
import pickle
import os,sys,re,shutil
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

with open('../input/book-len/book_dict.pickle', 'rb') as handle:
    book_dict = pickle.load(handle)
wrote_list = []
for author in book_dict:
    wrote_list.append( book_dict[author]['num'] )

plt.hist(wrote_list, bins=25)
plt.title("How many books each author wrote (total)")
plt.show()
plt.clf()

#Except 1
wrote_list.clear()
for author in book_dict:
    if( book_dict[author]['num'] > 2):
        wrote_list.append( book_dict[author]['num'] )

plt.hist(wrote_list, bins=20)
plt.title("How many books each author wrote (except 1)")
plt.show()
plt.clf()

## 3-c. How many lines exist in each book

In [ ]:

import pickle
import matplotlib.pyplot as plt
import sys
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

with open("../input/book-len/book_len.pickle", "rb") as handle:
    data = pickle.load( handle)

data= sorted(data)[:-1]

plt.hist(data, bins=100)
plt.title("How many lines exist in each book")
plt.show()
plt.clf()


There are about 20000 sentence by 3 authors in the original train data. Each author has about 6,000 sentences. 
So we choose books which has about 6,000(from 5,000 to 10,000) lines from the book corpus.

In addition, There are multi-topic books in the book corpus. we exclude these books for the convenience. we've got 112 authors. we exclude 5 authors among the authors: 'All authors named in this book', 'Rusty Fischer, author of Zombies Don’t Cry', 'Gavin Chappell James Rhodes Gav Roach', 'Author:', 'Serenity Valley Publishing'.

Finally, we have 110 authors ( 107 from the corpus + 3 from the original data ).

In [ ]:

import pickle
import matplotlib.pyplot as plt
import sys
import operator
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

with open("../input/book-len/author_dict.pickle", "rb") as handle:
    dict_ = pickle.load( handle)
book_dict= {}

new_dict = { "multi_topic": 0, "single_topic":0, "above5000_below10000":0}
topic_list = [0] * 10
count = []
author_list = []
for author in dict_:
    if(dict_[author]["total_sentence"] == dict_[author]["no_dup_sentence"]):
        new_dict["single_topic"] += 1
        count.append( dict_[author]["no_dup_sentence"])
        if( dict_[author]["no_dup_sentence"] > 5000 and dict_[author]["no_dup_sentence"] < 10000):
            new_dict["above5000_below10000"] += 1
            author_list.append(author)
    else:
        new_dict["multi_topic"] += 1
        
plt.bar(range(len(new_dict)), new_dict.values(), align='center')
plt.xticks(range(len(new_dict)), new_dict.keys())

plt.show()
print( "authors: %d" %(len( author_list)) )


## 3-d. How long each sentence is

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import sys
import operator


with open("../input/book-len/sent_length_dict.pickle", "rb") as handle:
    dict_ = pickle.load( handle)

        
plt.hist(dict_["original"], bins=1000)
plt.title("sentence length (Total) ")
# Show and clear plot again
plt.show()
plt.clf()       

list_ = []
for num in dict_["original"]:
    if( num < 500 and num > 30):
        list_.append(num)
plt.hist(list_, bins=50)
plt.title("sentence length ( 30< x < 500) ")
# Show and clear plot again
plt.show()
plt.clf()         

list_ = []
for num in dict_["original"]:
    if( num < 300 and num > 30):
        list_.append(num)
plt.hist(list_, bins=50)
plt.title("sentence length ( 30< x < 300) ")
# Show and clear plot again
plt.show()
plt.clf()   


We exclude sentences less than 30 characters, and more than 300 characters.

# 4. Training & Result

After the preprocessing step, we've got 464,500 sentence with 110 authors.
we trained our model with these data, and got a softmax value per each sentence.

To get author embedding, we sum all softmax vectors labeled with same author. We reduce the dimensionality of the vectors by T-SNE to 2D, and make scatter plots using pyplot. Each topic has different color.


![dd](https://www.kaggle.com/jinsooyeo/images/image-preview/sojong_result7.png)